In [86]:
import re
from pandas import read_csv

### Colunas
- Região
- Mês
- Alimentação
- Habitação
- Residência
- Vestuário
- Transportes
- Saúde
- Despesas Pessoais
- Educação
- Comunicação
- Alimentação (Peso)
- Habitação (Peso)
- Residência (Peso)
- Vestuário (Peso)
- Transportes (Peso)
- Saúde (Peso)
- Despesas Pessoais (Peso)
- Educação (Peso)
- Comunicação (Peso)

In [145]:
dados = read_csv('./dados.csv')

columns = list(dados.columns)[2:]

dados = dados[dados['Região'] != 'Grande Vitória (ES)']

dados['Ano'] =  dados['Mês'].apply(lambda mes: re.findall('\d{4}',mes) [0])

dados['Mês'] = dados['Mês'].apply(lambda mes: re.findall('\w+', mes)[0])

for column in columns:
    dados[column] = dados[column].apply(lambda value: value.replace(',', '.')).astype(float)
    

In [150]:
IPCA = dados['Alimentação'] * (dados['Alimentação (Peso)']/ 100)

for index in range(1,9):
    IPCA = IPCA + (dados[columns[index]] * (dados[columns[index + 9]] / 100))

dados['IPCA'] = IPCA / 9

In [156]:
agrupado_regiao = dados.groupby('Região')
agrupado_ano = dados.groupby('Ano')

agrupado_ano['IPCA'].sum()

Ano
2012     6.195962
2013     5.746852
2014     6.230474
2015    10.184569
2016     6.410034
2017     2.578996
2018     3.587676
2019     4.265869
Name: IPCA, dtype: float64